In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import pydeck as pdk

# 포스트 데이터 불러오기
df_post = pd.read_csv('../inputs/posts/itewon.csv', index_col=0)

# 텍스트 없으면 drop
df_post = df_post.dropna()

In [2]:
# 상점 데이터 불러오기
gdf_store_itewon = gpd.read_file('../inputs/stores/store_itewon.geojson')

# 상호명 없는 상점은 삭제
gdf_store_itewon = gdf_store_itewon.dropna(subset='상호명')

# 상호명과 geometry만 남기고 다 삭제 (현재 필요없는 컬럼들)
gdf_store_itewon = gdf_store_itewon.loc[:,['상호명', 'geometry']]

# 상호명: store_name으로 변경
gdf_store_itewon = gdf_store_itewon.rename(columns={'상호명':'store_name'})

In [3]:
gdf_store_itewon.shape

(3512, 2)

In [4]:
def counting_post(store_name):
    '''
    상점 이름을 input으로 받아 posting 갯수를 세주는 함수

    input:
        store_name (string): 상호명
    output:
        posting_count (int): 포스팅 갯수

    '''

    # 상호명이 한글자이면 그냥 0개로 취급
    if len(store_name) == 1:
        return 0

    # 포스트 중에서 상점 이름이 포함되어 있으면 좌표 추가
    posting_count = df_post.loc[df_post.loc[:,'text'].str.contains(store_name, regex=False)].shape[0]

    return posting_count


In [5]:
gdf_store_itewon.loc[:,'posting_count'] = gdf_store_itewon.loc[:,'store_name'].apply(counting_post)

In [6]:
# 포스팅 갯수 기준으로 정렬
gdf_store_itewon = gdf_store_itewon.sort_values(by='posting_count', ascending=False)

In [7]:
for i, row in gdf_store_itewon.iterrows():
    print(row['store_name'], row['posting_count'])
    if row['posting_count'] == 0:
        break

태원 687
크리스 178
갤러리 150
예인 146
맛집 118
리단 56
원타 55
하지 52
레스 48
하우스 46
포토 45
커피 41
스타일 33
이태원포토 31
로우 29
로우 29
이드 26
이드 26
코드 25
이클래스 24
메이 22
열심히 21
해방촌 21
앳더프론트 20
드레스코드 20
문화 20
취향 20
키키 20
정도 19
남산 18
오아시스 18
안주 18
코스 17
타임 16
버터 16
플레이 15
테오트리 12
포인트 11
로드 10
쿠키 10
니즈 10
일로 9
일로 9
마리 9
백사 8
하늘 8
바스 8
와이 8
반가 7
테라 7
깊은 7
약속 7
미용실 7
예요 7
공감 7
마라 6
러브하우스 6
인스테드 6
크로 6
이태원짐 6
헤미안 6
엘리 5
위드 5
베르 5
포즈 5
부다스벨리 5
마나 5
키스 5
브레드 5
부다스벨리 5
해피퍼피하우스 5
뉴메리크스튜디오 4
누에 4
가든 4
주쌀롱 4
소하 4
애플 4
필라테스 4
하하 3
오브제 3
카라 3
이태원클라쓰 3
그루브 3
현대 3
마당 3
여인 3
바리 3
소울트레인 3
프로포즈 3
클로버 3
우월 3
옥탑 3
챔프커피 3
스탠딩 3
코이 3
오브제 3
루아 3
시나몬 3
카라 3
헤미안커피바 3
테이프 3
타피 3
아카이브 3
새비지가든 3
이든 3
블랑 3
코끼리 3
챔프커피 3
챔프커피 3
콜리 2
이즈비 2
보라 2
파킨 2
술꼬마 2
오렌지 2
미자서울 2
이태원양꼬치 2
바베큐집 2
구구 2
욱이족발 2
와이아트스튜디오 2
아델 2
더서울 2
이태원맥주집 2
야상해 2
릴리 2
스태프온리 2
플랜트 2
수니 2
캐롤 2
아지트 2
초승달 2
레스트 2
스텝 2
로즈앤크라운 2
피치 2
레트로 2
우정 2
농담 2
요소 2
조아 2
왕타이 2
이민아헤어테크닉 2
스카이 2
이태원20 2
갤러리자유 2
피치 2
마메종394 2
퓨전 2
아지트 2
유니콘 2
노노샵 2
나니 2
블루도어북스 2
좋은비누연구소 2
신세계 2
온도 2
데모 2
한나 2
보리 2
매

In [8]:
list_stopwords = [
'태원',
'크리스',
'갤러리',
'예인',
'맛집',
'리단',
'원타',
'하지',
'레스',
'하우스',
'포토',
'커피',
'스타일',
'이태원포토'
]

In [9]:
# list_stopwords 안에 있는 상점들은 우선 데이터프레임에서 제외
gdf_store_itewon = gdf_store_itewon.loc[~gdf_store_itewon.loc[:,'store_name'].isin(list_stopwords)]

# 포스팅 수가 0인 것들도 데이터프레임에서 제외
gdf_store_itewon = gdf_store_itewon.loc[gdf_store_itewon.loc[:,'posting_count']>0]

In [10]:
gdf_store_itewon

,store_name,geometry,posting_count
3187,로우,POINT (126.98959 37.53339),29
377,로우,POINT (127.00190 37.53722),29
3189,이드,POINT (126.99640 37.53325),26
2858,이드,POINT (126.99916 37.53237),26
2685,코드,POINT (126.99523 37.53485),25
...,...,...,...
227,모하,POINT (126.99077 37.53963),1
1023,오띠젤리,POINT (126.98934 37.53325),1
2461,다모토리,POINT (126.99521 37.53390),1
866,모에드,POINT (126.99002 37.53343),1


In [11]:
# pydeck layer 설정

scatterplot_layer = pdk.Layer(
    'ScatterplotLayer',
    gdf_store_itewon,
    pickable=True,
    opacity=0.3,
    stroked=False,
    filled=True,
    radius_scale=2,
    get_position='geometry.coordinates',
    get_radius='posting_count',
    get_fill_color=[255,140,0]
)

In [12]:
# Set the viewport location
view_state = pdk.ViewState(latitude=37.5385, longitude=126.9925, zoom=15, bearing=0, pitch=0)

# Render
r = pdk.Deck(layers=[scatterplot_layer], initial_view_state=view_state, tooltip={"text": "{store_name}\n{posting_count}"})
r.to_html("../output/maps/itewon/scatterplot_layer.html")

In [13]:
heatmap_layer = pdk.Layer(
    "HeatmapLayer",
    data=gdf_store_itewon,
    opacity=0.5,
    get_position='geometry.coordinates',
    aggregation=pdk.types.String("MEAN"),
    get_weight="posting_count",
)

In [14]:
r = pdk.Deck(
    layers=[heatmap_layer],
    initial_view_state=view_state
)

r.to_html("../output/maps/itewon/heatmap_layer.html")